In [86]:
import numpy as np
import pandas as pd
from typing import List, Tuple, Callable

In [87]:
def euclidean_distance(x1: np.array, x2: np.array):
    return np.linalg.norm(x1 - x2)

In [88]:
class NearestNeighbourClassifier:
    def __init__(
            self,
            data: np.ndarray[np.ndarray],
            metric: Callable[[np.ndarray, np.ndarray], np.ndarray],
            labels: List[int],
            K: int
        ):
        """
        data: np.ndarray[np.ndarray] - array of data points
        metric: Callable[[np.ndarray, np.ndarray], np.ndarray] - distance metric
        labels: List[int] - list of labels for each data point, must start by 0
        K: int - number of neighbours to consider
        """
        self.metric = metric
        self.data = data
        self.labels = labels
        self.K = K
        self.n_classes = len(set(labels))
        self.n_points = data.shape[0]
        self.n_features = data.shape[1]
    
    def predict(self, x: np.array):
        """
        Predicts the label of a single data point
        x: np.array - data point
        """
        return np.argmax(self.get_probabilities(x))
    
    def decide(self, U: np.ndarray[np.ndarray[int]], x: np.array):
        """
        A method that return the action that maximise the expected utility.
        :param U: is a 2 dimentional array that indicated the utility of each action based on y.
                  example: U = np.array([[ 1  ,    -1000 ],
                                         [ -1 ,    0     ]])
                  so the U[1,0] indicated the utility of tanking the action a=1 based on y=0.
                  a/y | Cat   |   Dog  |
                  Cat |   1   |  -100  |
                  Dog |  -1   |    0   |
                  every row is an action and every column is a label.
        :param x: the test point.
        :return: the index of action that maximises the expected utility max_a E[U|a,x].
                 where E[U|a,x] = sum_y P(y|x) U(a,y). and the expected utility of this action.
        """
        # Need to use the get_probabilities function to return action with the highest expected utility
        # i.e. maximising tge sum_y U(y, a)P(y|x)

        n_action = U.shape[0] # number of actions (rows)
        n_labels = U.shape[1] # number of labels (columns)
        assert n_labels == self.n_classes # check that the number of labels in U is the same as the number of labels in the data

        probabilities = self.get_probabilities(x) # Array of probabilities of each label based on the K nearest neighbours.
        expected_utility = np.zeros(n_action)
        for action_index in range(n_action):
            # U[action_index] is the array of utilities of each label based on the action action_index
            expected_utility[action_index] = np.dot(probabilities, U[action_index]) # sum_y P(y|x) U(a,y), Transpose U to get the action in the first dimension
        
        argmax = np.argmax(expected_utility) # The index (in U) of the action that maximises the expected utility
        return argmax, expected_utility[argmax], expected_utility

    def get_probabilities(self, x: np.array):
        """
        A method that return the probability of each label based on the K nearest neighbours.
        :param x: the test point.
        :return: An array of probabilities of each label based on the K nearest neighbours.
                 The labels are the indices of the array.
        """
        # distances is an array of distances from x to the n first points in the data
        distances = np.zeros(self.n_points)
        for i in range(self.n_points):
            distances[i] = self.metric(x, self.data[i])

        # by sorting the distances on the indices, we can get the indices of the K nearest neighbours
        # because we built the distances array from the data array with the same indices
        indices = np.argsort(distances)

        # proportions is an array of the proportions of each label in the K nearest neighbours
        proportions = np.zeros(self.n_classes) 
        for i in range(self.K):
            k_neighbour = indices[i]
            k_neighbour_label = self.labels[k_neighbour] # get the label of the i-th nearest neighbour
            proportions[k_neighbour_label] += 1 # increment the count of the label 
        proportions /= self.K # divide by K to get the proportion of each label

        return proportions

In [89]:
n_population = 10
n_features = 4
n_real_classes = 3

X = np.random.uniform(size=(n_population, n_features))
y = np.random.randint(0, n_real_classes, size=(n_population,))

In [90]:
X

array([[0.59293773, 0.25794024, 0.85909889, 0.9723676 ],
       [0.36468528, 0.82158375, 0.67234266, 0.01478283],
       [0.99880261, 0.67962983, 0.34436478, 0.98292776],
       [0.65034929, 0.18206952, 0.503687  , 0.60613717],
       [0.19106074, 0.16514504, 0.3997362 , 0.36794466],
       [0.0389498 , 0.65125902, 0.39697232, 0.01631511],
       [0.83697755, 0.46170957, 0.20109038, 0.98361565],
       [0.50424643, 0.02057393, 0.02895283, 0.63990484],
       [0.28649619, 0.91445995, 0.17441769, 0.31655554],
       [0.41251552, 0.08839331, 0.18029088, 0.4084991 ]])

In [91]:
y

array([2, 1, 0, 1, 1, 2, 1, 0, 0, 0])

In [92]:
knn = NearestNeighbourClassifier(X, euclidean_distance, y, 3)

In [93]:
knn.get_probabilities(X[0])

array([0.        , 0.66666667, 0.33333333])

In [94]:
knn.predict(X[0])

1

In [95]:
DATA = pd.read_csv("data.csv")
# DATA = DATA.astype({
#     "Specialisation": "category",
#     "Eye Color": "category",
#     "Citiizenship": "category",
# })
DATA

,Name,Specialisation,Sex,Height (cm),Weight (kg),Age,Eye Color,Citiizenship,Exercise (h/week),Marital Status,...,Writing (0/1),Football (0/1),Basketball (0/1),Tennis (0/1),Swimming (0/1),Running (0/1),Mountain Sport (0/1),Biking (0/1),Weights (0/1),Other (0/1)
0,Christos,AI,m,178.0,80.0,48.0,Brown,Greece,7.0,Married,...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0
1,Arthur,Maths,m,177.0,77.0,23.0,Brown,Swiss,12.0,Single,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0
2,Ale,-,m,178.0,60.0,28.0,Brown,Cuban,5.0,married,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
3,Michèle,Data science,f,172.0,70.0,24.0,blue,Swiss,5.0,Single,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,Robin,Procrastination,M,177.0,63.0,25.0,Blue,Swiss,10.0,Single,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
5,Antoine,Maths,m,178.0,69.0,28.0,Blue,Swiss,7.0,Single,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
6,Soham,Data science,M,178.0,82.0,23.0,Black,Indian,NaN,Single,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
7,Amin,Data science,M,175.0,78.0,23.0,Brown,Bangladeshi,4.0,Single,...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0
8,Lien,Business Informatics,f,168.0,61.0,24.0,blue,Swiss,5.0,not married,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
9,Yvan,Geography,m,180.0,70.0,26.0,blue,Swiss,6.0,Single,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


In [96]:
X = DATA[["Height (cm)", "Weight (kg)"]].to_numpy()
y = DATA["Biking (0/1)"].to_numpy()
y[np.isnan(y)] = 0
X[np.isnan(X)] = 0
y = y.astype(int)

In [97]:
X

array([[178.,  80.],
       [177.,  77.],
       [178.,  60.],
       [172.,  70.],
       [177.,  63.],
       [178.,  69.],
       [178.,  82.],
       [175.,  78.],
       [168.,  61.],
       [180.,  70.],
       [155.,  52.],
       [165.,  54.],
       [170.,  85.],
       [170.,  65.],
       [183.,  73.],
       [180.,  95.],
       [177.,  70.],
       [  0.,   0.]])

In [98]:
X.shape

(18, 2)

In [99]:
y

array([0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0])

In [100]:
knn = NearestNeighbourClassifier(X, euclidean_distance, y, 3)

In [101]:
for t in range(X.shape[0]):
    x_t = X[t]
    p = knn.get_probabilities(x_t)
    print("Features:", x_t)
    print("Real class:", y[t])
    print("Score (probability of the real class to be choosen):", p[y[t]])
    print("Probabilities of every class (index = class number):", p)
    print("--------------------")

Features: [178.  80.]
Real class: 0
Score (probability of the real class to be choosen): 0.6666666666666666
Probabilities of every class (index = class number): [0.66666667 0.33333333]
--------------------
Features: [177.  77.]
Real class: 1
Score (probability of the real class to be choosen): 0.3333333333333333
Probabilities of every class (index = class number): [0.66666667 0.33333333]
--------------------
Features: [178.  60.]
Real class: 1
Score (probability of the real class to be choosen): 0.6666666666666666
Probabilities of every class (index = class number): [0.33333333 0.66666667]
--------------------
Features: [172.  70.]
Real class: 0
Score (probability of the real class to be choosen): 0.6666666666666666
Probabilities of every class (index = class number): [0.66666667 0.33333333]
--------------------
Features: [177.  63.]
Real class: 1
Score (probability of the real class to be choosen): 0.6666666666666666
Probabilities of every class (index = class number): [0.33333333 0.6

In [102]:
U = np.array([[1, -1000],
              [-1, 0]])

for x in X:
    print("x", x)
    print("Decision", knn.decide(U=U, x=x))
    print("")

x [178.  80.]
Decision (1, -0.6666666666666666, array([-332.66666667,   -0.66666667]))

x [177.  77.]
Decision (1, -0.6666666666666666, array([-332.66666667,   -0.66666667]))

x [178.  60.]
Decision (1, -0.3333333333333333, array([-6.66333333e+02, -3.33333333e-01]))

x [172.  70.]
Decision (1, -0.6666666666666666, array([-332.66666667,   -0.66666667]))

x [177.  63.]
Decision (1, -0.3333333333333333, array([-6.66333333e+02, -3.33333333e-01]))

x [178.  69.]
Decision (0, 1.0, array([ 1., -1.]))

x [178.  82.]
Decision (0, 1.0, array([ 1., -1.]))

x [175.  78.]
Decision (1, -0.6666666666666666, array([-332.66666667,   -0.66666667]))

x [168.  61.]
Decision (1, -0.6666666666666666, array([-332.66666667,   -0.66666667]))

x [180.  70.]
Decision (0, 1.0, array([ 1., -1.]))

x [155.  52.]
Decision (0, 1.0, array([ 1., -1.]))

x [165.  54.]
Decision (0, 1.0, array([ 1., -1.]))

x [170.  85.]
Decision (1, -0.6666666666666666, array([-332.66666667,   -0.66666667]))

x [170.  65.]
Decision (1, -